In [ ]:
# 运行 default 主机 hello
!ansible-playbook -i hosts -l default playbooks.yaml -vv --tags hello

In [ ]:
!ls

In [ ]:
!code --folder-uri vscode-remote://dev-container+docker//$HOME//project


ansible-playbook [core 2.16.7]
  config file = None
  configured module search path = ['/root/.ansible/plugins/modules', '/usr/share/ansible/plugins/modules']
  ansible python module location = /usr/local/lib/python3.12/site-packages/ansible
  ansible collection location = /root/.ansible/collections:/usr/share/ansible/collections
  executable location = /usr/local/bin/ansible-playbook
  python version = 3.12.3 (main, May 14 2024, 07:23:41) [GCC 12.2.0] (/usr/local/bin/python)
  jinja version = 3.1.4
  libyaml = True
No config file found; using defaults
Skipping callback 'default', as we already have a stdout callback.
Skipping callback 'minimal', as we already have a stdout callback.
Skipping callback 'oneline', as we already have a stdout callback.

PLAYBOOK: playbooks.yaml *******************************************************
1 plays in playbooks.yaml

PLAY [Hello] *******************************************************************

TASK [Gathering Facts] **************************